# NLP 실습 #3: N-gram Language model
황순원의 '소나기' 로 n-gram language model 학습하고 사용해보기
</br>
</br>

* 준비 : konlpy와 kenlm library 설치

* Step 1: Data preprocessing - tokenization

* Step 2: Language Model 생성

* Step 3: Binary file로 Model 변환

* Step 4: Model을 활용하여 시퀀스 scoring

* Step 5: 주어진 시퀀스에서 다음 형태소 예측

* Step 6: 주어진 시퀀스로 시작하는 문장을 예측

* Step 7: 다른 말뭉치로 학습한 모델과 비교

</br>
</br>

## 준비: konlpy와 kenlm library 설치

In [2]:
# 2. konlpy의 Kkma 형태소 분석기 import, 형태소 분석 예제

from konlpy.tag import Kkma

tokenizer = Kkma()
tokenizer.morphs("이 문장이 형태소 단위로 잘 출력되나요?")

['이', '문장', '이', '형태소', '단위', '로', '잘', '출력', '되', '나요', '?']

In [3]:
# 이전에 nltk 및 'punkt' 모듈을 다운로드 하지 않은 경우에 주석을 풀고 run

import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# 3. kenlm 설치

!wget -O - https://kheafield.com/code/kenlm.tar.gz |tar xz
!mkdir -p ./kenlm/build
%cd ./kenlm/build
!cmake ..
!make -j 4
!pip install https://github.com/kpu/kenlm/archive/master.zip
%cd /home/ch2_ngram_language_model

--2021-01-04 09:30:19--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491090 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 479.58K  1.43MB/s    in 0.3s    

2021-01-04 09:30:19 (1.43 MB/s) - written to stdout [491090/491090]

mkdir: cannot create directory ‘./kenlm/build’: File exists
/content/kenlm/build
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
-- Boost version: 1.65.1
-- Found the following Boost libraries:
--   program_options
--   system
--   thread
--   unit_test_framework
--   chrono
--   date_time
--   atomic
-- Configuring done
-- Generating done
-- Build files have been written to: /content/kenlm/build
[  2%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/bignum-dtoa.cc.o
[  2%] Building CXX object util/CMakeFiles/kenlm_u

## Step 1: Data preprocessing - tokenization
&emsp;&emsp; sonagi.doc의 내용을 문장 단위로 분리.

</br> &emsp;&emsp; <<Exercise 1>> 각 문장을 kkma 분석기로 형태소 단위로 분리.

</br> &emsp;&emsp; 형태소로 분석된 문장들과 vocab들을 sonagi.txt, sonagi.voc 파일로 작성.


In [5]:
# 4. nltk import하고 sonagi.doc을 읽어들임. nltk의 sent_tokenize를 이용하여 sonagi.doc의 내용을 문장 단위로 분리

from nltk.tokenize import sent_tokenize

lines = []
with open('sonagi.doc', 'rt', encoding='utf-8') as f:
    for line in f:
        lines.append(line)

sentences = []
for line in lines:
    sentences.extend(sent_tokenize(line))

In [6]:
print(sentences)

['소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(曾孫女)딸이라는 걸 알 수 있었다.', '소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다.', '서울서는 이런 개울물을 보지 못하기나 한 듯이.', '벌써 며칠째 소녀는, 학교에서 돌아오는 길에 물장난이었다.', '그런데, 어제까지 개울 기슭에서 하더니, 오늘은 징검다리 한가운데 앉아서 하고 있다.', '소년은 개울둑에 앉아 버렸다.', '소녀가 비키기를 기다리자는 것이다.', '요행 지나가는 사람이 있어, 소녀가 길을 비켜 주었다.', '다음 날은 좀 늦게 개울가로 나왔다.', '이 날은 소녀가 징검다리 한가운데 앉아 세수를 하고 있었다.', '분홍 스웨터 소매를 걷어올린 목덜미가 마냥 희었다.', '한참 세수를 하고 나더니, 이번에는 물 속을 빤히 들여다 본다.', '얼굴이라도 비추어 보는 것이리라.', '갑자기 물을 움켜 낸다.', '고기 새끼라도 지나가는 듯.', '소녀는 소년이 개울둑에 앉아 있는 걸 아는지 모르는지 그냥 날쌔게 물만 움켜 낸다.', '그러나, 번번이 허탕이다.', '그대로 재미있는 양, 자꾸 물만 움킨다.', '어제처럼 개울을 건너는 사람이 있어야 길을 비킬 모양이다.', '그러다가 소녀가 물 속에서 무엇을 하나 집어 낸다.', '하얀 조약돌이었다.', '그리고는 벌떡 일어나 팔짝팔짝 징검다리를 뛰어 건너간다.', '다 건너가더니만 홱 이리로 돌아서며,', '"이 바보."', '조약돌이 날아왔다.', '소년은 저도 모르게 벌떡 일어섰다.', '단발 머리를 나풀거리며 소녀가 막 달린다.', '갈밭 사잇길로 들어섰다.', '뒤에는 청량한 가을 햇살 아래 빛나는 갈꽃뿐.', '이제 저쯤 갈밭머리로 소녀가 나타나리라.', '꽤 오랜 시간이 지났다고 생각됐다.', '그런데도 소녀는 나타나지 않는다.', '발돋움을 했다.', '그러고도 상당한 시간이 지났다고 생각됐다.', '저 쪽 갈밭머리에 갈꽃이 한 옴큼 움직였다.', '소녀가 갈꽃을 안고 있었다.', '그리고, 이제는 천천한 걸

In [7]:
tokenizer.morphs("이 문장이 형태소 단위로 잘 출력되나요?")

['이', '문장', '이', '형태소', '단위', '로', '잘', '출력', '되', '나요', '?']

In [8]:
#########################################################################################################################
# 5. <<Exercise 1>> 주어진 sentences를 이용하여, sentences를 각 문장별로 형태소 분석한 morph_sentences를 implementation 해주세요.

# sentences: ['소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(曾孫女)딸이라는 걸 알 수 있었다.',
#             '소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다.',
#             ... ]
#########################################################################################################################


morph_sentences = []
for sentence in sentences:
    temp = ""
    morphs = tokenizer.morphs(sentence)
    for morph in morphs:
      temp = temp + " " + morph

    morph_sentences.append(temp.strip())


# <expected results>
# morph_sentences: ['소년 은 개울가 에서 소녀 를 보 자 곧 윤 초시 네 증손녀 ( 曾孫女 ) 딸 이 라는 것 을 알 ㄹ 수 있 었 다 .',
#                   '소녀 는 개울 에다 손 을 잠그 고 물장난 을 하 고 있 는 것 이 다 .',
#                   ... ]

In [9]:
# 6. 형태소 분석된 결과물 확인

print(sentences[0])
print(morph_sentences[0])
print()
print(sentences[1])
print(morph_sentences[1])

소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(曾孫女)딸이라는 걸 알 수 있었다.
소년 은 개울가 에서 소녀 를 보 자 곧 윤 초시 네 증손녀 ( 曾孫女 ) 딸 이 라는 것 을 알 ㄹ 수 있 었 다 .

소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다.
소녀 는 개울 에다 손 을 잠그 고 물장난 을 하 고 있 는 것 이 다 .


In [10]:
# 7. 형태소 분석된 각 sentence들을 한 줄에 한 문장씩 sonagi.txt 파일로 작성

with open('sonagi.txt', 'wt', encoding='utf-8') as f:
    for line in morph_sentences:
        f.write(line.strip())
        f.write('\n')

In [11]:
# 8. 형태소 분석된 각 sentence의 형태소들을 전부 set에 집어넣어 중복을 삭제, set에 들어있는 vocab을 한 줄에 한 형태소씩 .voc 파일로 작성

vocab = set()
for sentence in morph_sentences:
    temp = sentence.split()
    for t in temp:
        vocab.add(t)

print(vocab)

with open('sonagi.voc', 'wt', encoding='utf-8') as f:
    for line in vocab:
        f.write(line.strip())
        f.write('\n')

{'쳐다보', '머리', '소년', '아', '일어', '씹', '"', '유달리', '만지작거리', '바르', '한', '죽', '무슨', '를', '올르', '기울', '애', '한껏', '입', '가운데', '소녀', '돌아오', '데', '스치', '생채기', '얼굴', '옆', '오르', '수탉', '춤', '밭머리', '사뭇', '사내', '큰길', '저기', '시작', '가을걷이', '그러하', '곧', '잠자코', '째', '분홍', '줄', '공연히', '우리', '사이', '겠', '갈', '실패', '나풀거리', '주위', '제가', '제법', '벌', '입도', '잠방이', '주춤', '잘', '찌', '집', '자', '일어서', '자고', '그대로', '입히', '이것', '들국화', '다는', '증손', '말하', ')', '고마움', '라', '비탈지', '걸음', '도리어', '꺾이', '빛', '다시', '등나무', '조심', '범벅', '코피', '내려다보', '갈림길', '아래쪽', '열적', '복', '소리', '볼', '비치', '마리', '윤', '껑충거리', '이끌리', '내', '뵈', '날쌔', '밖', '가로막', '눕', '수박', '양산', '어지럽', '하', '딸', '그래', '달', '동무', '끼얹히', '변변히', '단맛', '내다보', '가져가', '야', '놓', '꽤', '질리', '손톱', '곳', '노랗', '어야지', '약도', '망그러지', '악상', '두', '여도', '새끼', '갈아입', '한결', '돌이', '잘못하', '나룻', '들길', '사잇길', '싱싱', '재미있', '순순히', '!', '쓰', '임', '패', '억새풀', '한테', '앞서', '순간', '걸어가', '진흙물', '제삿상', '잔등', '나', '학교', '소풍', '메밀밭', '지나', '날로', '고개', '건만', '서럽', '긋', '꽃가지', '흙탕물', '가깝', '집어던지', '찢어지',

In [12]:
! ls

buckwheat_flowers.arpa	buckwheat_flowers.txt  sample_data  sonagi.doc
buckwheat_flowers.bin	buckwheat_flowers.voc  sonagi.arpa  sonagi.txt
buckwheat_flowers.doc	kenlm		       sonagi.bin   sonagi.voc


## Step 2: Language model 생성


### &emsp;>> ! kenlm/build/bin/lmplz -o [N] < [txt file] > [model file]
&emsp;&emsp;[N] : 사용할 모델의 n-gram

</br> &emsp;&emsp;[txt file] : 학습(counting)에 사용할 문서 경로

</br> &emsp;&emsp;[model file] : 생성할 모델 파일명

In [13]:
# 9. kenlm 모델 build

! kenlm/build/bin/lmplz -o 3 <sonagi.txt> sonagi.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /content/sonagi.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 2514862080 bytes == 0x558f08f4c000 @  0x7f97f2c021e7 0x558f0779a7a2 0x558f0773551e 0x558f077142eb 0x558f07700066 0x7f97f0d9bbf7 0x558f07701baa
tcmalloc: large alloc 8382865408 bytes == 0x558f9eda8000 @  0x7f97f2c021e7 0x558f0779a7a2 0x558f077897ca 0x558f0778a208 0x558f07714308 0x558f07700066 0x7f97f0d9bbf7 0x558f07701baa
****************************************************************************************************
Unigram tokens 4557 types 944
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:11328 2:3799259392 3:7123611648
tcmalloc: large alloc 7123615744 bytes == 0x558f08f4c000 @  0x7f97f2c021e7 0x558f0779a7a2 0x558f077897ca 0x558f0778a208 0x558f077148d7 0x558f07700066 0x7f97f0d9bbf7 0x558f07701baa
tcmalloc: large alloc 3799261184 bytes == 0x5591930da000 @  0x7f97f2c021e7 

In [14]:
!ls


buckwheat_flowers.arpa	buckwheat_flowers.txt  sample_data  sonagi.doc
buckwheat_flowers.bin	buckwheat_flowers.voc  sonagi.arpa  sonagi.txt
buckwheat_flowers.doc	kenlm		       sonagi.bin   sonagi.voc


## Step 3: Binary file로 Model 변환
### &emsp;>> ! kenlm/build/bin/build_binary [model file] [bin model file]

&emsp;&emsp;[N] : n-gram

</br>&emsp;&emsp;[model file] : 이미 생성된 모델 파일명

</br>&emsp;&emsp;[bin model file] : binary로 생성할 모델 파일명

In [15]:
# 10. 생성된 kenlm 모델 파일 binary  file로 변환

! kenlm/build/bin/build_binary sonagi.arpa sonagi.bin

Reading sonagi.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [16]:
# 11. model 이 제대로 생성 되었는지 확인
# ! echo "원하는 문장" | kenlm/build/bin/query [(bin) model file]

! echo "소녀 가 소년 을" | kenlm/build/bin/query sonagi.bin
! echo "소녀 가 소년 을" | kenlm/build/bin/query sonagi.arpa

This binary file contains probing hash tables.
소녀=7 2 -1.0026854	가=72 3 -0.31729177	소년=3 1 -2.551379	을=23 1 -2.055063	</s>=2 1 -2.7724392	Total: -8.698858 OOV: 0
Perplexity including OOVs:	54.92520066053768
Perplexity excluding OOVs:	54.92520066053768
OOVs:	0
Tokens:	5
Name:query	VmPeak:42352 kB	VmRSS:6036 kB	RSSMax:6036 kB	user:0.003256	sys:0	CPU:0.00329119	real:0.000538565
Loading the LM will be faster if you build a binary file.
Reading sonagi.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
소녀=7 2 -1.0026854	가=72 3 -0.31729177	소년=3 1 -2.551379	을=23 1 -2.055063	</s>=2 1 -2.7724392	Total: -8.698858 OOV: 0
Perplexity including OOVs:	54.92520066053768
Perplexity excluding OOVs:	54.92520066053768
OOVs:	0
Tokens:	5
Name:query	VmPeak:42200 kB	VmRSS:6136 kB	RSSMax:6136 kB	user:0.002729	sys:0.002729	CPU:0.00550429	real:0.00285034


## Step 4: model을 활용하여 주어진 시퀀스 scoring

### &emsp;>> model.score('여름 이 었 다 . ', bos=True, eos=True)
### &emsp;>> model.score('여름 이 었 ', bos=True, eos=False)

In [17]:
# 12. 생성한 모델 불러오기
import kenlm

model_file = 'sonagi.bin' 
# model_file = 'sonagi.arpa' # arpa 모델도 사용가능하나, binary 파일이 속도가 좀더 빠름.

model = kenlm.Model(model_file)

In [18]:
# 13. 시퀀스 scoring 예시

# model.score('주어진 시퀀스', bos=True, eos=True)
# bos : 모델이 scoring하도록 주어진 시퀀스 내에서 sentence가 시작했는가
# eos : 모델이 scoring하도록 주어진 시퀀스 내에서 sentence가 종결되었는가

# 이미 완성한 시퀀스(문장)의 scoring을 위해서는 bos=True eos=True
# bos=True로 하면 모델이 시퀀스의 앞에 <s>를 추가하고,
# eos=True인 경우 모델이 시퀀스의 뒤에 </s>를 추가하여 score를 계산함

#따라서 bos=True, eos=True는 '<s> 원하는 시퀀스 </s>' 의 scoring을 한 것임.


#  "<s> 여름 이 었 다 . </s>" 를 scoring'
eos_score = model.score('여름 이 었 다 .', bos=True, eos=True)
print("<s> 여름 이 었 다 . </s> :", eos_score)

#  "<s> 여름 이 었" 을 scoring
no_eos_score = model.score('여름 이 었', bos=True, eos=False)
print("<s> 여름 이 었 :", no_eos_score)

<s> 여름 이 었 다 . </s> : -6.305783748626709
<s> 여름 이 었 : -6.258444786071777


## Step 5: 주어진 시퀀스에서 다음 형태소 예측

### &emsp;<< Exercise 2>> '소년 은 소녀 를' 다음에 올 가장 자연스러운 형태소를 찾아주세요.

In [19]:
# 14. vocab list 파일 불러오기

vocab_file = 'sonagi.voc'
vocab_list = []
with open(vocab_file, 'rt', encoding='utf-8') as f:
    for line in f:
        vocab_list.append(line.strip())

In [20]:
###########################################################################################################################
# 15. << Exercise 2 >> 주어진 시퀀스('소년 은 소녀 를') 다음의 형태소를 예측 : 주어진 시퀀스에 voc list의 형태소 하나를 합쳐서 score를 계산

# 주어진 시퀀스에 preprocessing하며 만든 sonagi.voc 파일의 형태소를 하나씩 붙여서 가장 score가 높은 형태소를 찾음
# voc = ['가', '만', '사업', ...] 이라면,
#'소년 은 소녀 가' 의 점수, '소년 은 소녀 만', '소년 은 소녀 사업 ', ... 등의 score를 계산하고 그 중 가장 높은 score를 지닌 형태소를 찾음
############################################################################################################################

best_vocab = None
best_score = None

for vocab in vocab_list:
    sequence_cand = "소년 은 소녀 를 " + vocab
    score_cand = model.score(sequence_cand, bos=True, eos=False)

    if best_vocab is None:
      best_score = score_cand
      best_vocab = vocab

    if best_score < score_cand:
      best_score = score_cand
      best_vocab = vocab

sequence_cand = "소년 은 소녀 를"
score_cand = model.score(sequence_cand, bos=True, eos=True)

if best_score < score_cand:
  best_score = score_cand
  best_vocab = '</s>'

print("Best vocab: ", best_vocab)

Best vocab:  보


## Step 6: 주어진 시퀀스로 시작하는 가장 자연스러운 문장 예측

#### </br>&emsp;<< Exercise 3 >> Step 5와 loop문을 사용하여,
#### </br> &emsp;'소년 은 소녀 를' 으로 시작하는 가장 자연스러운 문장을 찾아주세요.

In [21]:
###############################################################################################################################
# 16. << Exercise 3 >> 15를 auto-regressive하게 loop 문으로 반복하여 주어진 시퀀스 '소년 은 소녀 를' 로 시작한 문장이 어떻게 끝날지 예측해주세요.

# 1) 주어진 시퀀스에서 문장이 종결될 경우의 score를 계산
# 2) 주어진 시퀀스에서 문장이 종결되지 않을 경우에 그 다음에 올 가장 자연스러운 형태소와 score를 계산
# 3) 만약 1)보다 2)의 score가 더 높다면, 시퀀스에 2)에서 찾아낸 형태소를 추가하고, 다시 1)부터 반복
# 4) 만약 1)의 score가 2)의 score 보다 더 높다면, 그대로 시퀀스를 마무리 함
###############################################################################################################################

def predict(model_path, voc_path):
    # load language model with KenLM
    model = kenlm.Model(model_path)
    print("Language Model at " + model_path + " loaded.")

    # load vocabulary (or load word list)
    voc = []
    file_reader = open(voc_path, "r", encoding='utf-8')
    for line in file_reader:
        voc.append(line.strip())
    print("Vocabulary at " + voc_path + " loaded\n")

    sequence = input('Sequence: ')

    print("Given sequence: " + sequence)

    # prediction loop
    loop_condition = True
    while loop_condition:
        best_score = None 
        best_vocab = None 

        for vocab in voc:
            sequence_cand = sequence + " " + vocab
            sequence_score = model.score(sequence_cand, bos=True, eos=False)

            if best_score is None:
                best_score = sequence_score
                best_vocab = vocab

            if best_score < sequence_score:
                best_score = sequence_score
                best_vocab = vocab

        eos_sentence = sequence
        eos_score = model.score(eos_sentence, bos=True, eos=True)
        
        if eos_score < best_score:
            sequence = sequence + " " + best_vocab
      
        else:
            loop_condition = False


    print("Result: " + sequence)

    return

In [22]:
# 17. 16을 실행 

model_path = 'sonagi.arpa'
voc_path = 'sonagi.voc'
predict(model_path, voc_path)

Language Model at sonagi.arpa loaded.
Vocabulary at sonagi.voc loaded

Sequence: 소년 은 소녀 를
Given sequence: 소년 은 소녀 를
Result: 소년 은 소녀 를 보 게 되 었 다 .


## Step 7: 다른 말뭉치로 학습한 모델과 비교

#### </br> &emsp;<< Exercise 3 >> Step 1, 2, 3, 4, 6을 참고하여, 
#### </br> &emsp;bucketwheat_flowers.doc를 사용한 모델을 생성하여 
#### </br> &emsp;주어진 시퀀스('소년 은 소녀 를', '개울가 가', '여름 에')로 시작하는 가장 자연스러운 문장들을 찾고, 
#### </br> &emsp;sonagi.doc를 사용한 경우와 비교해주세요.

In [23]:
####################################################################################################################
# 18. << Exercise 4 >> : bucketwheat_flowers.doc를 사용한 모델을 생성하여 sonagi.doc를 사용한 경우와 주어진 시퀀스에 대한 output을 비교해주세요.
# 주어진 시퀀스: '소년 은 소녀 를', '개울가 가', '여름 에'

# sonata.doc를 data preprocessing하여 sonata.txt, sonata.voc를 생성.
# sonata.txt, sonata.voc를 이용하여 모델을 생성, load하여서
# 같은 문장에 대해서 sonagi corpus를 이용한 경우와 sonata corpus를 이용한 경우에 output이 어떠한지 출력
####################################################################################################################

# Data preprocessing. 형태소 분하여 buckwheat_flowers.txt, buckwheat_flowers.voc 작성
b_lines = []
with open('buckwheat_flowers.doc', 'rt', encoding='utf-8') as b_f:
    for b_line in b_f:
        b_lines.append(b_line)

b_sentences = []
for line in b_lines:
    b_sentences.extend(sent_tokenize(line))

b_morph_sentences = []
for sentence in b_sentences:
    temp = ""
    for morpheme in tokenizer.morphs(sentence):
      temp = temp + " " + morpheme
    b_morph_sentences.append(temp.strip())

with open('buckwheat_flowers.txt', 'wt', encoding='utf-8') as f:
    for line in b_morph_sentences:
        f.write(line.strip())
        f.write('\n')

b_vocab = set()
for sentence in b_morph_sentences:
    temp = sentence.split()
    for t in temp:
        b_vocab.add(t)

with open('buckwheat_flowers.voc', 'wt', encoding='utf-8') as f:
    for line in b_vocab:
        f.write(line.strip())
        f.write('\n')


# n-gram language model을 buckwheat_flowers로 학습

! kenlm/build/bin/lmplz -o 3 <buckwheat_flowers.txt> buckwheat_flowers.arpa
! kenlm/build/bin/build_binary buckwheat_flowers.arpa buckwheat_flowers.bin

=== 1/5 Counting and sorting n-grams ===
Reading /content/buckwheat_flowers.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 2514862080 bytes == 0x55abd2048000 @  0x7f266b9281e7 0x55abd120a7a2 0x55abd11a551e 0x55abd11842eb 0x55abd1170066 0x7f2669ac1bf7 0x55abd1171baa
tcmalloc: large alloc 8382865408 bytes == 0x55ac67ea4000 @  0x7f266b9281e7 0x55abd120a7a2 0x55abd11f97ca 0x55abd11fa208 0x55abd1184308 0x55abd1170066 0x7f2669ac1bf7 0x55abd1171baa
****************************************************************************************************
Unigram tokens 4807 types 1193
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:14316 2:3799258368 3:7123609600
tcmalloc: large alloc 7123615744 bytes == 0x55abd2048000 @  0x7f266b9281e7 0x55abd120a7a2 0x55abd11f97ca 0x55abd11fa208 0x55abd11848d7 0x55abd1170066 0x7f2669ac1bf7 0x55abd1171baa
tcmalloc: large alloc 3799261184 bytes == 0x55ae5c1d6000 @  0x7

In [24]:
# 19. 18을 실행

model_path2 = 'buckwheat_flowers.arpa'
voc_path2 = 'buckwheat_flowers.voc'
predict(model_path2, voc_path2)

Language Model at buckwheat_flowers.arpa loaded.
Vocabulary at buckwheat_flowers.voc loaded

Sequence: 개울가 가
Given sequence: 개울가 가
Result: 개울가 가 어디 없이 하얗 ㄴ 것 이 었 다 .
